*Last edit: 5th August 2024*

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib
# import seaborn as sns; sns.set_theme()

# import gurobipy as gp
# from gurobipy import GRB

from functions import *

import os
import json
# import re
import pprint

### Analysing the Objective Function in the Robust Portfolio Optimization Problem

In [2]:
# seed = 339
# num_runs = 1000

# for data in ['GICS','Fama-French']:
#     if data == 'GICS':
#         unit_list = ['monthly']
#         num_sectors_list = [11]
#     else:
#         unit_list = ['monthly','daily']
#         num_sectors_list = [5,10,12,17]
#     for units in unit_list:
#         for num_sectors in num_sectors_list:

#             save_units = units.capitalize()
            
#             if units == 'daily':
#                 N = [100]
#                 num_years=10
#                 num_units=int(num_years*365*(5/7))
#             elif units=='monthly':
#                 N = [24]
#                 num_years=30
#                 num_units=int(num_years*12)

#             true_mu, true_cov = true_mean_cov(data,units,num_sectors,num_units)
            
#             np.random.seed(seed=seed)
            
#             mu_hat = {}
            
#             for run in range(num_runs):
#                 mu_hat[run] = {}
#                 for n in N:
#                     mu_hat[run][n] = np.mean(simulate(true_mu, true_cov, N), axis=0)                        
            
#             identity = 0
#             sigma = 0
#             inv_sigma = 1
            
#             assert identity + sigma + inv_sigma == 1, 'Choose exactly one type of estimation error matrix'    
            
#             if sigma:
#                 # topx = num_sectors // 2
#                 # bottomx = (num_sectors // 2) + 1
#                 topx = num_sectors
#                 bottomx = 0
#                 power_list = [2]
#             elif inv_sigma:
#                 topx = num_sectors
#                 bottomx = 0
#                 power_list = [1,2,4,10]
#             else:
#                 power_list = [None]    
                
#             for power in power_list:

#                 if data == 'Fama-French':
#                     path = os.path.join('Simulated Robust','Fama-French',str(num_sectors)+' Sectors',save_units)
#                     analysis_path = os.path.join('plots','Simulated Robust','Objective Analysis','Fama-French',str(num_sectors)+' Sectors',save_units)
#                 elif data == 'GICS':
#                     path = os.path.join('Simulated Robust','GICS')
#                     analysis_path = os.path.join('plots','Simulated Robust','Objective Analysis','GICS')
                
#                 if sigma or inv_sigma:
#                     power_label = ['Zeroth','','Squared','Cubed','Fourth','Fifth','Sixth','Seventh','Eighth','Ninth','Tenth'][power]
                
#                     diag = np.diag(true_cov)
#                     topindices = np.argpartition(diag, -topx)[-topx:]
#                     bottomindices = np.argpartition(diag, bottomx)[:bottomx]
                
#                 Xi = np.eye(num_sectors)
                
#                 if identity:
#                     topx = 'NA'
#                     path = os.path.join(path,'Xi_identity') 
#                     analysis_path = os.path.join(analysis_path,'Xi_identity')
#                 elif sigma:
#                     path = os.path.join(path,'Xi_Sigma'+str(power_label))
#                     analysis_path = os.path.join(analysis_path,'Xi_Sigma'+str(power_label))
#                     for idx in topindices:
#                         Xi[idx][idx] = (diag[idx])**(power/2)
#                     for idx in bottomindices:
#                         Xi[idx][idx] = np.mean((diag)**(power/2))
#                 elif inv_sigma:
#                     path = os.path.join(path,'Xi_invSigma'+str(power_label))
#                     analysis_path = os.path.join(analysis_path,'Xi_invSigma'+str(power_label))
#                     for idx in topindices:
#                         Xi[idx][idx] = 1/(diag[idx]**(power/2))
#                     for idx in bottomindices:
#                         Xi[idx][idx] = 1/np.mean((diag)*(power/2))
                
#                 robust_portfolios = {}
#                 save_dir = os.listdir(os.path.join('save',path))
#                 for file in save_dir:
#                     if 'robust_portfolios' in file:
#                         with open(os.path.join('save',path,file)) as f: 
#                             portfolios_temp = json.load(f)
#                         robust_portfolios.update(portfolios_temp)
                
#                 plots_dir = os.listdir(os.path.join('plots',path))
#                 results_df = pd.DataFrame()
#                 for file in plots_dir:    
#                     if 'csv' in file:        
#                         if results_df.empty:
#                             results_df = pd.read_csv(os.path.join('plots',path,file), header=None)
#                             results_df.columns = results_df.iloc[0]
#                             results_df = results_df.drop([0])
#                         else: 
#                             temp_df = pd.read_csv(os.path.join('plots',path,file), header=None)
#                             temp_df.columns = temp_df.iloc[0]
#                             temp_df = temp_df.drop([0])
#                             results_df = pd.concat([results_df, temp_df], ignore_index=True)                    
#                 results_df.columns = ['kappa'] + list(results_df.columns[1:])
                
#                 obj_analysis = {}
#                 for v in list(results_df.columns)[1:]:
                
#                     best_kappa = results_df['kappa'].loc[int(results_df.idxmax().loc[v])]
                        
#                     lin_terms = []
#                     quad_terms = []
#                     kappa_quad_terms = []
                    
#                     for run in range(num_runs):        
#                         if str(best_kappa) not in robust_portfolios.keys():        
#                             if type(best_kappa) == float or type(best_kappa) == np.float64:
#                                 best_kappa = int(best_kappa)
#                             elif type(best_kappa) == int or type(best_kappa) == np.int64:
#                                 best_kappa = float(best_kappa)        
                        
#                         x = robust_portfolios[str(best_kappa)][str(run)][str(N[0])][str(v)]
                        
#                         lin_terms.append(mu_hat[run][N[0]]@x)
#                         quad_terms.append(np.sqrt(x@Xi@x))
#                         kappa_quad_terms.append(best_kappa*np.sqrt(x@Xi@x))
                        
#                     avg_lin_term = float(np.mean(lin_terms))
#                     avg_quad_term = float(np.mean(quad_terms))
#                     avg_kappa_quad_term = float(np.mean(kappa_quad_terms))
                    
#                     obj_analysis[v] = [float(best_kappa), avg_lin_term, avg_quad_term, avg_kappa_quad_term, avg_lin_term/avg_quad_term,
#                                        avg_lin_term/avg_kappa_quad_term]
                
#                 final_df = pd.DataFrame(obj_analysis, index = ['Best kappa', 'Linear Term', 'Quadratic Term', 
#                                                                'Best kappa * Quadratic Term', 'Linear Term/Quadratic Term',
#                                                                'Linear Term/Best kappa * Quadratic Term'])
#                 Xi_label = os.path.split(path)[-1]
#                 file_name = 'ObjectiveAnalysis '+data+' '+str(num_sectors)+'-sectors '+units+' '+Xi_label+'.csv'
            
#                 incumbent_path = []
#                 for directory in analysis_path.split(os.sep):
#                     incumbent_path.append(directory)
#                     if not os.path.exists(os.path.join(*incumbent_path)):
#                         os.mkdir(os.path.join(*incumbent_path))
                
#                 final_df.to_csv(os.path.join(analysis_path,file_name),index=True)

***

### Picking $\kappa$ using our algorithm

* Picking $\kappa$
    1. Compute $\hat \mu \bar x$ where $\bar x$ is the equal weight portfolio
    2. Compute $\sqrt{\bar {\bar x}^\top \Xi \bar {\bar x}}$ where $\bar {\bar x} = (\frac{1}{2}, \frac{1}{2(n-1)},\ldots)$ where the $\xi_1$ is the smallest.
    3. Pick $\kappa = \frac{\hat \mu^\top \bar x}{3 \sqrt{\bar {\bar x}^\top \Xi \bar {\bar x}}}$
    4. Find $x^R$ with this $\kappa$
    5. Stop if $2 \leq \frac{\hat \mu^\top x^R}{\kappa \sqrt{{x^R}^\top \Xi x^R}} \leq 4$
    6. Else $\kappa \leftarrow \frac{\hat \mu^\top x^R}{3 \sqrt{{x^R}^\top \Xi x^R}}$


In [3]:
def choose_kappa(data, units, num_sectors, save = False, quad_choice = 2):
    save_units = units.capitalize()
                
    seed = 339
    np.random.seed(seed=seed)
    num_runs = 1000
    
    if units == 'daily':
        N = [100]
        num_years=10
        num_units=int(num_years*365*(5/7))
    elif units=='monthly':
        N = [24]
        num_years=30
        num_units=int(num_years*12)

    true_mu, true_cov = true_mean_cov(data,units,num_sectors,num_units)    
            
    mu_hat = {}
    
    for run in range(num_runs):
        mu_hat[run] = {}
        for n in N:
            mu_hat[run][n] = np.mean(simulate(true_mu, true_cov, N), axis=0)
    
    min_var_portfolio, min_var_obj = min_variance(true_cov)

    v_begin = min_var_obj
    v_end = np.diag(true_cov).max()
    v_array = np.linspace(v_begin, v_end, 10)
    v_array = [float(np.format_float_positional(v_array[i], precision=3, unique=False, fractional=False, trim='k')) for i in range(len(v_array))]

    if v_array[0] < min_var_obj:
        v_array[0] = float(str(v_array[0])[:-1] + str(int(str(v_array[0])[-1])+1))
    
    # if units=='monthly':
    #     v_factor = 1
    # elif units=='daily':
    #     v_factor = 0.1
    
    # v_begin = next(x[1] for x in enumerate([i*2.5*v_factor for i in range(11)]) if x[1] > min_var_obj)
    # v_end = v_begin+(20*v_factor)
    # precision = 2.5*v_factor
    # num_v = (v_end - v_begin) / precision
    # v_array = np.linspace(v_begin, v_end, int(num_v) + 1).tolist()
    
    if data == 'Fama-French':
        path = os.path.join('Simulated Robust','Fama-French',str(num_sectors)+' Sectors',save_units)
    elif data == 'GICS':
        path = os.path.join('Simulated Robust','GICS')
    
    identity = 0
    sigma = 0
    inv_sigma = 1
    
    assert identity + sigma + inv_sigma == 1, 'Choose exactly one type of estimation error matrix'    
             
    if sigma:
        topx = num_sectors
        bottomx = 0
        power = 2
        # topx = num_sectors // 2
        # bottomx = (num_sectors // 2) + 1
    elif inv_sigma:
        topx = num_sectors
        bottomx = 0
        power = 10

    if sigma or inv_sigma:
        power_label = ['Zeroth','','Squared','Cubed','Fourth','Fifth','Sixth','Seventh','Eighth','Ninth','Tenth'][power]
        
        diag = np.diag(true_cov)
        topindices = np.argpartition(diag, -topx)[-topx:]
        bottomindices = np.argpartition(diag, bottomx)[:bottomx]
    
    Xi = np.eye(num_sectors)
    
    if identity:
        topx = 'NA'
        path = os.path.join(path,'Xi_identity') 
    elif sigma:
        path = os.path.join(path,'Xi_Sigma'+str(power_label))
        for idx in topindices:
            Xi[idx][idx] = (diag[idx])**(power/2)
        for idx in bottomindices:
            Xi[idx][idx] = np.mean((diag)**(power/2))
    elif inv_sigma:
        path = os.path.join(path,'Xi_invSigma'+str(power_label))
        for idx in topindices:
            Xi[idx][idx] = 1/(diag[idx]**(power/2))
        for idx in bottomindices:
            Xi[idx][idx] = 1/np.mean((diag)*(power/2))
    
    avg_lin_term = np.abs(np.mean([mu_hat[i][N[0]] for i in range(len(mu_hat))]))

    if quad_choice == 1:
        quad_term_portfolio = [1/(2*(num_sectors-1)) for i in range(num_sectors)]
        quad_term_portfolio[np.diag(Xi).argmin()] = 1/2
    elif quad_choice == 2:
        inv_diagXi = 1/np.diag(Xi)
        quad_term_portfolio = inv_diagXi/inv_diagXi.sum()  # alternate choice for \bar\bar x 
                                                                     # where we normalize Xi    
        # print(np.diag(Xi))
        # print(quad_term_portfolio)
    quad_term = np.sqrt(quad_term_portfolio@Xi@quad_term_portfolio)
    
    kappas = {}
    for v in v_array:
        kappas[v] = float(avg_lin_term/(3*quad_term))
    
    robust_portfolios = {}
        
    # print('Computing Robust Portfolios\n')
    
    stop_criteria = [False for i in range(len(v_array))]
    
    while sum(stop_criteria) != len(v_array):
        # print('\nCurrent choice for kappas:')
        # pprint.pprint(kappas)
                
        robust_portfolios = {}
        for run in range(num_runs):
            robust_portfolios[run] = {}
            # if run % 100 == 0:
            #     print('\t\tOn run ', run)
            
            robust_portfolios[run] = {}                
            for v in v_array:                
                robust_estimated_portfolio, robust_estimated_obj = robust(mu_hat[run][N[0]], true_cov, Xi, v, kappas[v])
                robust_portfolios[run][v] = robust_estimated_portfolio
                
        for i in range(len(v_array)):
            avg_lin_term = np.abs(np.mean([mu_hat[run][N[0]]@robust_portfolios[run][v_array[i]] for run in range(num_runs)]))
            avg_quad_term = np.mean([np.sqrt(robust_portfolios[run][v_array[i]]@Xi@robust_portfolios[run][v_array[i]]) for run in range(num_runs)])
            ratio = avg_lin_term/(kappas[v_array[i]]*avg_quad_term)
            # print('Ratio for', v_array[i], '% risk is', ratio)
            if ratio <= 2 or ratio >= 4:
                kappas[v_array[i]] = float(avg_lin_term/(3*avg_quad_term))
            else:
                stop_criteria[i] = True
    #     print('Stopping criteria:', stop_criteria, '\n')
    
    print('\nFinal kappa choices for',data,num_sectors,'sectors',units,'data:')
    pprint.pprint(kappas)

    if save:
        with open(os.path.join('save',path,'heuristic_kappas (top{}, N={}).json'.format(topx,N)), 'w') as f: 
            json.dump(kappas, f, ensure_ascii=False, indent=4)
    
    return kappas

#### Results for first choice of quadratic term 
$\sqrt{\bar {\bar x}^\top \Xi \bar {\bar x}}$ where $\bar {\bar x} = (\frac{1}{2}, \frac{1}{2(n-1)},\ldots)$ where the $\xi_1$ is the smallest.

In [4]:
# final_kappas = {}
# best_final_kappas = {}
# for data in ['GICS','Fama-French']:
#     final_kappas[data] = {}
#     best_final_kappas[data] = {}
#     if data == 'GICS':
#         unit_list = ['monthly']
#         num_sectors_list = [11]
#     else:
#         unit_list = ['monthly','daily']
#         num_sectors_list = [5,10,12,17]
#     for units in unit_list:
#         final_kappas[data][units] = {}
#         best_final_kappas[data][units] = {}
#         for num_sectors in num_sectors_list:
#             all_kappas = choose_kappa(data, units, num_sectors, num_years=30, quad_choice=1)
#             final_kappas[data][units][num_sectors] = all_kappas
#             best_final_kappas[data][units][num_sectors] = final_kappas[data][units][num_sectors][-1]
#             print('\n')

In [5]:
# pprint.pprint(best_final_kappas)

#### Results for second choice of quadratic term 
$\sqrt{\bar {\bar x}^\top \Xi \bar {\bar x}}$ where $\bar {\bar x}$ is a normalization of $\text{diag}(\sigma^4)$.

In [6]:
final_kappas = {}
# best_final_kappas = {}
for data in ['GICS','Fama-French']:
    final_kappas[data] = {}
    # best_final_kappas[data] = {}
    if data == 'GICS':
        unit_list = ['monthly']
        num_sectors_list = [11]
    else:
        unit_list = ['monthly','daily']
        num_sectors_list = [5,10,12,17]
    for units in unit_list:
        final_kappas[data][units] = {}
        # best_final_kappas[data][units] = {}
        for num_sectors in num_sectors_list:
            kappas = choose_kappa(data, units, num_sectors, save=True, quad_choice=2)
            final_kappas[data][units][num_sectors] = kappas
            # all_kappas = choose_kappa(data, units, num_sectors, save=False, quad_choice=2)
            # final_kappas[data][units][num_sectors] = all_kappas
            # best_final_kappas[data][units][num_sectors] = final_kappas[data][units][num_sectors][-1]
            print('\n')

Considering GICS data
Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-03

Final kappa choices for GICS 11 sectors monthly data:
{11.5: 1271.8826258960485,
 16.0: 4566.153389635443,
 20.6: 8728.377437589743,
 25.1: 13959.114873990846,
 29.7: 13959.114873990846,
 34.3: 13959.114873990846,
 38.8: 13959.114873990846,
 43.4: 13959.114873990846,
 47.9: 13959.114873990846,
 52.5: 13959.114873990846}


Considering Fama-French 5-sector monthly data

Final kappa choices for Fama-French 5 sectors monthly data:
{14.4: 733.8814895719537,
 17.1: 1287.170898937114,
 19.8: 1762.9480645938945,
 22.5: 3277.5725706633443,
 25.1: 3277.5725706633443,
 27.8: 3277.5725706633443,
 30.5: 3277.5725706633443,
 33.2: 3277.5725706633443,
 35.9: 3277.5725706633443,
 38.6: 3277.5725706633443}


Considering Fama-French 10-sector monthly data

Final kappa choices for Fama-French 10 sectors monthly data:
{11.3: 670.8334258241042,
 18.4: 2822.91662410803,
 25.5: 6988.557491003389,

In [7]:
pprint.pprint(final_kappas)

{'Fama-French': {'daily': {5: {1.03: 0.04452141917398256,
                               1.11: 0.09915986539957586,
                               1.19: 0.09915986539957586,
                               1.27: 0.09915986539957586,
                               1.35: 0.09915986539957586,
                               1.43: 0.09915986539957586,
                               1.52: 0.09915986539957586,
                               1.6: 0.09915986539957586,
                               1.68: 0.09915986539957586,
                               1.76: 0.09915986539957586},
                           10: {0.85: 0.027276931861199828,
                                1.23: 0.20253863260340543,
                                1.62: 0.3566767286720732,
                                2.02: 0.7669320867607484,
                                2.41: 0.7669320867607484,
                                2.8: 0.7669320867607484,
                                3.19: 0.7669320867607484,
            

In [8]:
with open(os.path.join('save','Simulated Robust','Heuristic_Kappas','Xi_invSigmaTenth.json'), 'w') as f: 
    json.dump(final_kappas, f, ensure_ascii=False, indent=4)

***

### Notes

* The linear term and quadratic term are for the associated best $\kappa$ value for each value of $v$. That is, they are the terms $\hat \mu^\top x$ and $\sqrt{x^\top \Xi x}$ for the portfolio $x$ found using with the best $\kappa$ (determined as that with the largest percentage gap closed from the choices of $\kappa$ in our grid search).